In [1]:
import plotly.graph_objects as go
import numpy as np
import folium
from shapely import wkt, geometry
import json
from openrouteservice import client
import pandas as pd
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import os

In [2]:
api_key = ''

ziele=pd.read_csv("msg_standorte_deutschland.csv",index_col=0)

In [3]:

ziele

,msg Standort,Straße,Hausnummer,PLZ,Ort,Breitengrad,Längengrad
Nummer,,,,,,,
1,Ismaning/München (Hauptsitz),Robert-Bürkle-Straße,1,85737,Ismaning,48.229035,11.686153
2,Berlin,Wittestraße,30,13509,Berlin,52.580911,13.293884
3,Braunschweig,Mittelweg,7,38106,Braunschweig,52.278748,10.524797
4,Bretten,Edisonstraße,2,75015,Bretten,49.032767,8.698372
5,Chemnitz,Zwickauer Straße,16a,9122,Chemnitz,50.829383,12.914737
6,Düsseldorf,Gladbecker Straße,3,40472,Düsseldorf,51.274774,6.794912
7,Essen,Am Thyssenhaus,1.3,45128,Essen,51.450577,7.008871
8,Frankfurt,Mergenthalerallee,73-75,65760,Eschborn,50.136479,8.570963
9,Görlitz,Melanchthonstraße,19,2826,Görlitz,51.145511,14.970028


In [4]:
longitude=ziele['Längengrad'].tolist()
latitude=ziele['Breitengrad'].tolist()

#API request for distance matrix on a openrouteservice requires list of coordinates with longitude and latitude

coordinates=[]
coordinates.extend([list(a) for a in zip(longitude, latitude)]) 
print(coordinates)   

[[11.686153, 48.229034999999996], [13.293884, 52.580911], [10.524797, 52.278748], [8.698372, 49.032767], [12.914736999999999, 50.829383], [6.794911999999999, 51.274774], [7.008871000000001, 51.450577], [8.570963, 50.136479], [14.970028, 51.145511], [9.986065, 53.557577], [9.769706, 52.337987], [11.399106, 48.784417], [6.913119, 50.886726], [7.322185, 52.519154], [7.61428, 51.969304], [11.017403999999999, 49.429596000000004], [13.453256, 48.571989], [7.936808999999999, 53.537779], [8.325873, 48.126258], [9.161239, 48.694648], [8.649036, 49.295010999999995]]


In [5]:
try: 
    with open('matrix.json') as f:
        jsonmatrix=json.load(f)
        
    matrixcombined = pd.read_csv('matrixcombined.csv')
    print("CSV file loaded to pandas Dataframe")
except IOError:
    
    print("file not found, making API request")
    
    import requests

    body = {"locations":coordinates,"metrics":["duration","distance"],"units":"km"}
    #your API-key must be inserted
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': api_key,
        'Content-Type': 'application/json; charset=utf-8'
        }
    try:
        call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    except:
        print("\n Request failed")
        
    print("\n Request successfull")
    print(call.text)
    
    jsonmatrix=call.json()
    with open('matrix.json', 'w') as f:
        json.dump(jsonmatrix, f,indent=3)
        
    print("Json file saved")
    
    matrixcombined =pd.json_normalize(jsonmatrix)
    matrixcombined.to_csv('matrixcombined.csv',index=False)
    print("\n CSV file created")

CSV file loaded to pandas Dataframe


In [6]:
durations=pd.json_normalize(jsonmatrix,record_path='durations')
distances=pd.json_normalize(jsonmatrix,record_path='distances')
destinations=pd.json_normalize(jsonmatrix,record_path='destinations')
sources=pd.json_normalize(jsonmatrix,record_path='sources')
#duration of each route in hours rounded to 2 decimal points
#durations
durations_t=durations.transpose()

durations_t
#durations/=3600
#durations.round(2)
#in order to work with integers multiply by 100, which after solving will be scaled back
#durations*=100
#durations.round(0)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.00,17636.87,17451.20,9778.52,11723.31,18876.64,19603.69,12259.10,16848.84,23401.51,...,2945.34,17571.40,21855.75,20298.31,5212.46,6144.78,24185.86,11588.22,7270.20,10946.99
1,17967.82,0.00,8350.35,21726.71,10028.93,18085.35,17439.71,17163.79,11063.29,9494.38,...,16390.59,19371.41,16753.22,15886.00,14536.20,19615.24,15735.92,24160.18,20415.99,19781.97
2,18059.91,8277.05,0.00,16693.27,10058.47,10888.81,10243.17,11766.83,12883.19,7104.12,...,16482.69,12174.87,9556.68,8689.46,14628.30,19707.34,8931.16,21846.12,18101.92,14385.02
3,9823.34,21447.76,16929.07,0.00,15534.20,12045.71,12772.75,5456.36,20659.73,21848.29,...,10855.38,10740.47,16486.71,14482.53,8880.96,14686.24,19764.32,6652.23,2908.04,2797.65
4,11712.53,10024.92,9717.82,15471.42,0.00,17482.92,16598.75,12840.35,6268.52,15668.13,...,10135.30,17848.91,18120.69,16676.98,8280.91,12577.28,17495.18,17904.89,14160.70,14037.16
5,18783.65,17704.58,10735.41,12104.23,17599.17,0.00,1276.36,7771.58,21285.54,12156.81,...,17206.42,3103.91,5115.74,4106.72,14680.19,20391.22,9515.60,17257.08,13512.89,9795.98
6,19342.21,17198.32,10229.15,12756.59,16777.88,1328.82,0.00,8281.45,20464.25,11643.88,...,17764.98,3756.27,5305.30,3593.78,15238.75,20949.78,9002.67,17909.45,14165.25,10448.34
7,11969.69,16841.43,11921.83,5409.03,13109.18,7541.29,8268.33,0.00,17825.12,16841.05,...,10392.46,6236.05,11982.29,9475.29,7866.23,13577.26,14757.08,10561.88,6817.69,3100.78
8,17012.80,10984.15,12565.51,20771.70,6467.13,21134.04,20249.87,17697.59,0.00,18515.82,...,15435.58,22088.60,20968.38,20101.16,13581.19,17877.56,20342.87,23205.17,19460.98,19337.44
9,24227.91,9474.22,7432.96,21903.69,16226.47,12677.55,11897.55,16977.25,19009.58,0.00,...,22650.68,14229.20,9973.00,9261.96,20796.29,25875.33,7928.85,27056.54,23312.35,19595.44


In [7]:
#distances in km
#distances
#transposed matrix of duration so that Colums Indicies represent start and row represent finish

distances_t=distances.transpose()

distances_t

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.00,574.59,595.53,277.44,376.37,607.52,624.55,395.91,552.25,781.63,...,74.05,569.47,701.45,657.51,157.90,179.38,798.34,328.79,219.28,329.25
1,574.73,0.00,233.56,674.63,283.62,564.81,530.97,542.96,312.48,272.44,...,512.98,584.53,491.84,472.38,447.36,632.01,481.88,742.73,648.66,638.79
2,595.54,232.71,0.00,466.56,304.09,342.92,309.08,333.59,414.02,197.78,...,533.79,362.64,269.95,250.50,468.17,652.82,265.60,632.00,537.93,429.42
3,279.79,674.01,469.70,0.00,475.79,342.51,359.55,137.93,651.66,623.36,...,288.55,304.47,493.63,407.85,250.87,446.50,610.17,155.43,61.36,42.42
4,376.06,283.38,304.00,475.95,0.00,544.90,519.50,389.90,177.59,490.10,...,314.30,515.35,562.27,517.09,248.69,391.83,557.92,544.06,449.99,441.95
5,608.09,563.78,343.89,342.83,546.44,0.00,28.02,230.63,685.81,399.24,...,546.34,68.19,162.30,123.60,450.29,666.06,326.12,508.27,414.19,305.68
6,625.76,530.51,310.62,358.50,521.01,27.98,0.00,247.85,660.39,372.61,...,564.00,83.86,157.82,96.97,467.96,683.73,299.49,523.93,429.86,321.35
7,395.28,542.59,336.64,137.56,393.02,224.98,242.02,0.00,559.37,490.31,...,333.52,186.94,376.10,274.79,237.47,453.25,477.11,303.00,208.92,100.41
8,551.79,320.92,414.76,651.69,178.39,684.51,659.11,558.58,0.00,600.86,...,490.04,708.40,673.03,653.58,424.43,567.57,668.68,719.80,625.72,617.69
9,782.08,272.71,199.01,622.98,490.63,399.30,372.36,490.01,594.48,0.00,...,720.33,435.78,247.47,279.19,654.71,839.36,217.86,788.42,694.35,585.84


In [8]:
#germany map
map1 = folium.Map(tiles='Stamen Toner', location=([51.136479,8.570963]), zoom_start=6)
distances_t[1][2]

232.71

In [9]:
#markers on a map

for i in range(0,len(ziele)):
    folium.Marker([ziele.iloc[i]['Breitengrad'], ziele.iloc[i]['Längengrad']], popup=ziele.iloc[i]['msg Standort']).add_to(map1)

#def getDistance(from_id, to_id):
#    return int(distances_t[from_id][to_id])
#getDistance(1,2)

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distances_t[from_node][to_node])




 

In [10]:

tsp_size = len(ziele)
num_routes = 1
start = 0 # arbitrary start location


optimal_coords = []


manager = pywrapcp.RoutingIndexManager(tsp_size,num_routes, start)
routing = pywrapcp.RoutingModel(manager)


transit_callback_index = routing.RegisterTransitCallback(distance_callback)


routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


In [11]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [12]:

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    global route 
    route=[]
    print('Objective: {} Km'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        route.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    route.append(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}Km\n'.format(route_distance)
    
    


In [13]:
solution = routing.SolveWithParameters(search_parameters)
routeduration=0
if solution:
    print_solution(manager, routing, solution)
    
for i in range(0,21):
    stretchduration=int(durations_t[route[i]][route[i+1]])
    routeduration+=stretchduration
print(routeduration)
print('Duration: {} Hours'.format(round(routeduration/3600,3)))

Objective: 3185 Km
Route for vehicle 0:
 0 -> 11 -> 15 -> 18 -> 19 -> 3 -> 20 -> 7 -> 12 -> 5 -> 6 -> 14 -> 13 -> 17 -> 9 -> 10 -> 2 -> 1 -> 8 -> 4 -> 16 -> 0

117717
Duration: 32.699 Hours


In [14]:
optimal_coordinates=[coordinates[x] for x in route]
optimal_coordinates

[[11.686153, 48.229034999999996],
 [11.399106, 48.784417],
 [11.017403999999999, 49.429596000000004],
 [8.325873, 48.126258],
 [9.161239, 48.694648],
 [8.698372, 49.032767],
 [8.649036, 49.295010999999995],
 [8.570963, 50.136479],
 [6.913119, 50.886726],
 [6.794911999999999, 51.274774],
 [7.008871000000001, 51.450577],
 [7.61428, 51.969304],
 [7.322185, 52.519154],
 [7.936808999999999, 53.537779],
 [9.986065, 53.557577],
 [9.769706, 52.337987],
 [10.524797, 52.278748],
 [13.293884, 52.580911],
 [14.970028, 51.145511],
 [12.914736999999999, 50.829383],
 [13.453256, 48.571989],
 [11.686153, 48.229034999999996]]

In [15]:
from openrouteservice import directions
import os.path
from openrouteservice import client, places


clnt = client.Client(key=api_key)


def style_function(color):
    return lambda feature: dict(color=color,
                              weight=3,
                              opacity=1)

# See what a 'random' tour would have been
#Coordinates.append(Coordinates[0])

request = {'coordinates': optimal_coordinates,
           'profile': 'driving-car',
           'geometry': 'true',
           'format_out': 'geojson',
           'instructions': 'false'          
          }



# And now the optimal route
try:
    
    with open('optimalroute.json') as json_file:
        optimal_route = json.load(json_file)
    print("JSON file found")
except:
    
    optimal_route = clnt.directions(**request)
    print("No file. API Request sent.")
    
    jsonroute=json.dumps(optimal_route)
    with open('optimalroute.json', 'w') as o:
        json.dump(jsonroute, o,indent=3)
       
folium.features.GeoJson(data=optimal_route,name='Optimal Route',style_function=style_function('#6666ff'),overlay=True).add_to(map1)
map1.add_child(folium.map.LayerControl())


        
map1

JSON file found


In [16]:
route1=[x+1 for x in route]
citiesorder=ziele.reindex(route1)
s=[x for x in range(0,22)]
citiesorder.insert(loc=0, column='Rheienfolge', value=s)
citiesorder

,Rheienfolge,msg Standort,Straße,Hausnummer,PLZ,Ort,Breitengrad,Längengrad
Nummer,,,,,,,,
1,0,Ismaning/München (Hauptsitz),Robert-Bürkle-Straße,1,85737,Ismaning,48.229035,11.686153
12,1,Ingolstadt,Pascalstraße,4,85057,Ingolstadt,48.784417,11.399106
16,2,Nürnberg,Südwestpark,60,90449,Nürnberg,49.429596,11.017404
19,3,St. Georgen,Leopoldstraße,1,78112,St. Georgen,48.126258,8.325873
20,4,Stuttgart,Humboldtstraße,35,70771,Leinfelden-Echterdingen,48.694648,9.161239
4,5,Bretten,Edisonstraße,2,75015,Bretten,49.032767,8.698372
21,6,Walldorf,Altrottstraße,31,69190,Walldorf,49.295011,8.649036
8,7,Frankfurt,Mergenthalerallee,73-75,65760,Eschborn,50.136479,8.570963
13,8,Köln/Hürth,Max-Planck-Straße,40,50354,Hürth,50.886726,6.913119


In [17]:
citiesorder.to_csv("Solution.csv")